<a href="https://colab.research.google.com/github/verma7/AdventOfCode2021/blob/master/RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# RL

In [2]:
!pip install tf-agents[reverb]
!pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 96.2 MB/s eta 0:00:00
   ━━

In [3]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [4]:
!pip install galois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 54.6 MB/s eta 0:00:00


In [94]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import random
import galois
import pandas as pd
import math

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

from tf_agents.policies import random_py_policy
from tf_agents.policies import random_tf_policy
from tf_agents.metrics import py_metrics
from tf_agents.metrics import tf_metrics
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.drivers import dynamic_step_driver

from tf_agents import specs
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.replay_buffers import py_uniform_replay_buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

from tf_agents.train.utils import strategy_utils


In [122]:
def get_num_ones(w):
    "Returns a vector containing the number of ones in the bit representation in each element of the GF(2^w) field."
    N = 2 ** w
    GF = galois.GF(N)
    num_ones = np.zeros(N)
    for i in range(N):
        for j in range(w):
            v = np.multiply(GF(i), GF(2 ** j))
            num_ones[i] += bin(v).count("1")
    return num_ones

def count_ones(GC, n, m, ones):
    sum_ones = 0
    for i in range(n):
        for j in range(m):
            sum_ones += ones[int(GC[i][j])]
    return sum_ones

def swap(x, y, state_x, state_y, state):
  xi = 0
  for i in range(len(state)):
    if state[i] == state_x:
      if xi == x:
        break
      xi += 1

  xj = 0
  for j in range(len(state)):
    if state[j] == state_y:
      if xj == y:
        break
      xj += 1

  tmp = state[i]
  state[i] = state[j]
  state[j] = tmp

def normalize_bitmatrix(GF, GC, m, ONES):
  # Make the first row 1s by dividing all the columns by its first element.
  M = np.zeros((m, m))
  for i in range(m):
      for j in range(m):
          if GC[0][j] == 0:
              continue
          M[i][j] = np.divide(GF(int(GC[i][j])), GF(int(GC[0][j])))
#    print(M)

  # For each row, pick one of the elements that minimizes the number of ones and divide the row by it.
  for i in range(1, m):
      min_ones = 1e8
      best_C = None

      for j in range(m):
          if M[i][j] == 0:
              continue

          C = np.zeros(m)
          sum_ones = 0
          C_ones = []
          for k in range(m):
              C[k] = np.divide(GF(int(M[i][k])), GF(int(M[i][j])))
              sum_ones += ONES[int(C[k])]
              C_ones.append(ONES[int(C[k])])
          if sum_ones < min_ones:
              min_ones = sum_ones
              best_C = C
      for j in range(m):
          if best_C is not None:
              M[i][j] = best_C[j]
  return M

def good_cauchy_matrix(GF, m, N, state):
  GC = np.zeros((m, m))
  x = 0
  y = 0
  for i in range(N):
      if state[i] == 0:
        y = 0
        for j in range(N):
            if state[j] == 1:
              sum = GF(i) + GF(j)
              if sum == 0:
                  continue
              GC[x][y] = sum ** -1
              y += 1
        x += 1
  return GC

def simulate_action(action, m, N, state, GF, ONES):
  max_action = m * m + 2 * m * N - 1
  if action < 0 or action > max_action:
    raise ValueError(f'`action` should be between [0, {max_action}), but is {action}')
  elif action < m * m:
    x = action // m
    y = action % m
    swap(x, y, 0, 1, state)
  elif action < m * m + m * N:
    denom = m * N
    offset = m * m
    x = (action - offset) // denom
    y = (action - offset) % denom
    swap(x, y, 0, 2, state)
  elif action < m * m + 2 * m * N:
    denom = m * N
    offset = m * m + m * N
    x = (action - offset) // denom
    y = (action - offset) % denom
    swap(x, y, 1, 2, state)
  else:
    raise ValueError(f'`action` should be between [0, {max_action}), but is {action}')

  GC = good_cauchy_matrix(GF, m, N, state)
  NGC = normalize_bitmatrix(GF, GC, m, ONES)
  return count_ones(NGC, m, m, ONES)

def stochastic_hill_climbing_action(w, m, N, state, num_ones):
  """Returns an action"""
  max_action = m * m + 2 * m * N - 1
  GF = galois.GF(N)
  ONES = get_num_ones(w)

  GC = good_cauchy_matrix(GF, m, N, state)
  NGC = normalize_bitmatrix(GF, GC, m, ONES)
  actions = [action for action in range(max_action + 1)]
  random.shuffle(actions)
  for action in actions:
    state_copy = state.copy()
    sim_num_ones = simulate_action(action, m, N, state_copy, GF, ONES)
    reward = num_ones - sim_num_ones
    # print(f'action = {action}, reward = {reward}, prev num_ones = {num_ones}, sim num ones = {sim_num_ones}')
    if reward > 0:
      return action
  return -1

def simulated_annealing_action(w, m, N, state, num_ones, temperature):
  """Returns an action"""
  max_action = m * m + 2 * m * N - 1
  GF = galois.GF(N)
  ONES = get_num_ones(w)

  GC = good_cauchy_matrix(GF, m, N, state)
  NGC = normalize_bitmatrix(GF, GC, m, ONES)
  actions = [action for action in range(max_action + 1)]
  random.shuffle(actions)

  weights = np.zeros(max_action + 1)
  for action in actions:
    state_copy = state.copy()
    sim_num_ones = simulate_action(action, m, N, state_copy, GF, ONES)
    reward = num_ones - sim_num_ones
    weights[action] = reward
    # print(f'action = {action}, reward = {reward}, prev num_ones = {num_ones}, sim num ones = {sim_num_ones}')
    metropolis = math.exp(reward / temperature)
    if reward > 0 or random.random() < metropolis:
      return action
  return -1

def construct_state(vec, m, N):
  state = [0] * N
  for i in vec[:m]:
      state[i] = 0
  for i in vec[m:2*m]:
      state[i] = 1
  for i in vec[2*m:]:
      state[i] = 2
  return state

In [178]:
def create_population(population_size, m, N):
  pop = []
  vec = [ i for i in range(N)]
  for i in range(population_size):
    random.shuffle(vec)
    state = construct_state(vec, m, N)
    pop.append(state)
  return pop

def compute_fitness(individual, GF, m, N, w, ONES):
  GC = good_cauchy_matrix(GF, m, N, individual)
  NGC = normalize_bitmatrix(GF, GC, m, ONES)
  ones = count_ones(NGC, m, m, ONES)
  return m * m * w * w - ones, ones

def crossover(x, y, m):
  # print(f'Crossover: {x}, {y}')

  assert len(x) == len(y)
  N = len(x)
  new = np.full(N, 2)
  remaining = set()

  num_0 = 0
  num_1 = 0
  for i in range(N):
    if x[i] == y[i]:
        new[i] = x[i]
        if x[i] == 0:
          num_0 += 1
        elif x[i] == 1:
          num_1 += 1
    elif x[i] in [0, 1] or y[i] in [0, 1]:
      remaining.add(i)

  remaining = list(remaining)
  random.shuffle(remaining)

  k = 0
  for i in range(num_0, m):
      new[remaining[k]] = 0
      k += 1
  for j in range(num_1, m):
      new[remaining[k]] = 1
      k += 1

  return new

def genetic_algorithm(w, m):
  N = 2 ** w
  population_size = 5
  num_generations = 1000
  mutation_rate = 0.1

  GF = galois.GF(N)
  ONES = get_num_ones(w)

  population = create_population(population_size, m, N)
  print(population)

  best_ones = 1e8
  for i in range(num_generations):
    fitnesses = []
    for individual in population:
      fitness, ones = compute_fitness(individual, GF, m, N, w, ONES)
      if ones < best_ones:
        best_ones = ones
        print(f'generation = {i}, best ones = {best_ones}')
      fitnesses.append(fitness)
#      print(f'Individual: {individual}, fitness: {fitness}, ones: {ones}')

      if random.random() < mutation_rate:
        action = random.randint(0, m * m + 2 * m * N - 1)
        # print(f'Action: {action}')
        # print(f'Old individual = {individual}')
        simulate_action(action, m, N, individual, GF, ONES)
        # print(f'New individual = {individual}')

    new_population = []
    weights = np.array(fitnesses)
    weights = weights / np.sum(weights)
    indices = np.arange(population_size)
    for i in range(population_size):
      selected = np.random.choice(indices, p=weights, size=2, replace=False)
      new_population.append(crossover(population[selected[0]], population[selected[1]], m))
    population = new_population

genetic_algorithm(8, 6)

[[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2], [2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [115]:
class CauchyEnv(py_environment.PyEnvironment):

  def __init__(self, W, m, vec, num_ones_list):
    self._W = W
    self._N = 2 ** self._W
    self._m = m
    self._GF = galois.GF(self._N)
    self._ONES = get_num_ones(self._w)
    self._num_ones_list = num_ones_list

    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int64, minimum=0, maximum=self._m * self._m + 2 * self._m * self._N - 1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(self._N, ), dtype=np.int32, minimum=0, maximum=2, name='observation')
    self._initialize(vec)

  def _initialize(self, vec):
    self._best_num_ones = 1e8
    self._vec = vec
    self._state = construct_state(self._vec, self._m, self._N)
    self._GC = good_cauchy_matrix(self._GF, self._m, self._N, self._state)
    NGC = normalize_bitmatrix(self._GF, self._GC, self._m, self._ONES)
    self._num_ones = count_ones(NGC, self._m, self._m, self._ONES)
    self._best_num_ones = self._num_ones
    self._time_steps_without_improvement = 0

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._initialize(self._vec)
    return ts.restart(np.array(self._state, dtype=np.int32))

  def _step(self, action):
    new_num_ones = simulate_action(action, self._m, self._N, self._state, self._GF, self._ONES)
    reward = self._num_ones - new_num_ones
    self._num_ones = new_num_ones
    self._num_ones_list.append(new_num_ones)
    if new_num_ones < self._best_num_ones:
      self._best_num_ones = new_num_ones
      # print(f'Best num ones = {new_num_ones}')
      self._time_steps_without_improvement = 0
    else:
      self._time_steps_without_improvement += 1
    # print(f'Time steps without improvement: {self._time_steps_without_improvement}, best num ones = {self._best_num_ones}')
    if self._time_steps_without_improvement == 10:
      self._time_steps_without_improvement = 0
      self._initialize(self._vec)
      return ts.termination(np.array(self._state, dtype=np.int32), reward)
    else:
      return ts.transition(np.array(self._state, dtype=np.int32), reward=reward, discount=1.0)

In [106]:
w = 5
m = 5
N = 2 ** w
# Random restart stochastic hill climbing
vec = [ i for i in range(N)]
random.shuffle(vec)


max_episodes = 10
num_episodes = 0

num_ones = []
env = CauchyEnv(w, m, vec, num_ones)
tf_env = tf_py_environment.TFPyEnvironment(env)

while (num_episodes < max_episodes):
  time_step = tf_env.reset()
  num_episodes += 1
  num_ones.clear()

  while not time_step.is_last():
    action = stochastic_hill_climbing_action(env._w, env._m, env._N, env._state, env._num_ones)
    if action == -1:
      break
    time_step = tf_env.step(action)

  if num_ones:
    print(num_ones)
    print(f'Min num_ones: {np.min(num_ones)}')

[224.0, 214.0, 212.0, 209.0, 208.0]
Min num_ones: 208.0
[213.0, 207.0, 204.0, 198.0, 194.0, 185.0]
Min num_ones: 185.0
[213.0, 207.0, 205.0, 197.0]
Min num_ones: 197.0
[224.0, 214.0, 213.0, 211.0, 209.0, 198.0]
Min num_ones: 198.0
[224.0, 204.0, 203.0]
Min num_ones: 203.0
[222.0, 214.0, 212.0, 209.0, 208.0, 206.0]
Min num_ones: 206.0
[213.0, 207.0, 198.0, 193.0, 190.0, 185.0]
Min num_ones: 185.0
[213.0, 207.0, 204.0, 202.0, 194.0, 189.0]
Min num_ones: 189.0
[213.0, 207.0, 198.0, 195.0, 194.0, 189.0]
Min num_ones: 189.0
[213.0, 207.0, 204.0, 202.0, 189.0]
Min num_ones: 189.0


In [113]:
w = 5
m = 5
N = 2 ** w

max_iter = 1000
max_episodes = 10
num_episodes = 0

while (num_episodes < max_episodes):
  # vec = [ i for i in range(N)]
  # random.shuffle(vec)

  num_ones = []
  env = CauchyEnv(w, m, vec, num_ones)
  tf_env = tf_py_environment.TFPyEnvironment(env)

  time_step = tf_env.reset()
  num_episodes += 1
  num_ones.clear()

  i = 0
  for i in range(max_iter):
    temperature = 100 - (i + 1.0)/max_iter*100 + 1
    action = simulated_annealing_action(env._w, env._m, env._N, env._state, env._num_ones, temperature)
    if action == -1:
      break
    time_step = tf_env.step(action)

  if num_ones:
    print(num_ones)
    print(f'Min num_ones: {np.min(num_ones)}')

[213.0, 213.0, 213.0, 207.0, 238.0, 228.0, 218.0, 228.0, 230.0, 244.0, 230.0, 230.0, 244.0, 244.0, 213.0, 213.0, 207.0, 207.0, 238.0, 238.0, 207.0, 238.0, 222.0, 222.0, 236.0, 236.0, 236.0, 213.0, 213.0, 207.0, 238.0, 238.0, 238.0, 238.0, 207.0, 238.0, 238.0, 207.0, 207.0, 238.0, 213.0, 213.0, 213.0, 213.0, 207.0, 207.0, 207.0, 238.0, 207.0, 238.0, 238.0, 238.0, 223.0, 223.0, 232.0, 213.0, 230.0, 231.0, 215.0, 215.0, 232.0, 237.0, 224.0, 224.0, 224.0, 227.0, 226.0, 202.0, 202.0, 202.0, 202.0, 202.0, 202.0, 202.0, 239.0, 238.0, 238.0, 234.0, 213.0, 213.0, 207.0, 213.0, 213.0, 207.0, 207.0, 239.0, 239.0, 241.0, 239.0, 241.0, 239.0, 213.0, 207.0, 207.0, 238.0, 228.0, 237.0, 219.0, 215.0, 232.0, 235.0, 232.0, 232.0, 224.0, 224.0, 224.0, 224.0, 214.0, 250.0, 226.0, 226.0, 239.0, 239.0, 226.0, 225.0, 225.0, 234.0, 225.0, 224.0, 214.0, 224.0, 214.0, 224.0, 214.0, 214.0, 224.0, 224.0, 214.0, 214.0, 214.0, 224.0, 230.0, 220.0, 220.0, 225.0, 222.0, 229.0, 222.0, 222.0, 229.0, 222.0, 229.0, 222.0

KeyboardInterrupt: 

In [15]:
w = 3
m = 3
N = 2 ** w
vec = [ i for i in range(N)]
random.shuffle(vec)

R = np.zeros((N, N))
num_ones = []
num_actions = m * m + 2 * m * N
for i in range(num_actions):
  environment = CauchyEnv(w, m, vec, num_ones)
  time_step = environment.reset()
  time_step = environment.step(i)
  R[i//N][i%N] = time_step.reward

print(num_ones)
for i in range(N):
  print(' '.join([f"{R[i][j]:3.0f}" for j in range(N)]))
print(np.min(num_ones))

[32.0, 36.0, 34.0, 34.0, 36.0, 35.0, 34.0, 33.0, 32.0, 36.0, 32.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0]
  3  -1   1   1  -1   0   1   2
  3  -1   3   1   1   1   1   1
  1   1   1   1   1   1   1   1
  1   1   1   1   1   1   1   1
  1   1   1   0   0   0   0   0
  0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0
32.0


In [ ]:
W = 4
N = 5
vec = [ i for i in range(2 ** W)]
random.shuffle(vec)
env = CauchyEnv(W, N, vec, num_ones)
tf_env = tf_py_environment.TFPyEnvironment(env)

tf_policy = random_tf_policy.RandomTFPolicy(action_spec=tf_env.action_spec(),
                                            time_step_spec=tf_env.time_step_spec())


num_episodes = tf_metrics.NumberOfEpisodes()
env_steps = tf_metrics.EnvironmentSteps()
max_return = tf_metrics.MaxReturnMetric()
replay_buffer = []
observers = [num_episodes, env_steps, max_return, replay_buffer.append]
driver = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env, tf_policy, observers, num_episodes=100)

# Initial driver.run will reset the environment and initialize the policy.
final_time_step, policy_state = driver.run()

print('final_time_step', final_time_step)
print('Number of Steps: ', env_steps.result().numpy())
print('Number of Episodes: ', num_episodes.result().numpy())
print('Max Return: ', max_return.result().numpy())
print('Min num_ones: ', np.min(num_ones))


for epsilon in np.arange(0, 1.1, 0.1):
  num_ones = []
  environment = CauchyEnv(W, N, vec, num_ones)
  tf_env = tf_py_environment.TFPyEnvironment(environment)

  q_net = q_network.QNetwork(
      tf_env.time_step_spec().observation,
      tf_env.action_spec(),
      fc_layer_params=(100,))

  agent = dqn_agent.DqnAgent(
      tf_env.time_step_spec(),
      tf_env.action_spec(),
      q_network=q_net,
      epsilon_greedy=epsilon,
      optimizer=tf.compat.v1.train.AdamOptimizer(1e-4))

  collect_steps_per_iteration = 100
  avg_return_metric = tf_metrics.AverageReturnMetric()
  avg_episode_length_metric = tf_metrics.AverageEpisodeLengthMetric()
  observers = [] #avg_return_metric, avg_episode_length_metric]
  collect_op = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env,
    agent.collect_policy,
    observers=observers,
    num_episodes=collect_steps_per_iteration).run()

  print(num_ones)
  print(f"Best num_ones with epsilon greedy={epsilon} is {np.min(num_ones)}")
  # print(f"Avg return = {avg_return_metric.result().numpy()}")
  # print(f"Avg episode length = {avg_episode_length_metric.result().numpy()}")


In [ ]:
learning_rate = 1e-5

W = 8
N = 10
vec = [ i for i in range(2 ** W)]
random.shuffle(vec)
num_ones = []
environment = CauchyEnv(W, N, vec, num_ones)
tf_env = tf_py_environment.TFPyEnvironment(environment)

eval_num_ones = []
eval_environment = CauchyEnv(W, N, vec, eval_num_ones)
tf_eval_env = tf_py_environment.TFPyEnvironment(eval_environment)

fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(tf_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate))

replay_buffer_capacity = 10000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_capacity)
replay_buffer.clear()

# Add an observer that adds to the replay buffer:
replay_observer = [replay_buffer.add_batch]

collect_steps_per_iteration = 1000
collect_op = dynamic_episode_driver.DynamicEpisodeDriver(
  tf_env,
  agent.collect_policy,
  observers=replay_observer,
  num_episodes=collect_steps_per_iteration).run()

print(num_ones)
print(f"Best num_ones = {np.min(num_ones)}")

In [ ]:
print(agent.collect_policy._epsilon)

In [ ]:
def compute_min_ones(environment, policy, num_ones, num_episodes=10):
  total_num_ones = 0
  actions = []
  for _ in range(num_episodes):
    time_step = environment.reset()
    num_ones.clear()
    actions.clear()
    rewards_list = []
    rewards = 0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      rewards_list = np.concatenate((rewards_list, time_step.reward.numpy()))
      rewards += time_step.reward
      actions.append(action_step.action.numpy())
    print(actions)
    print(num_ones)
    print(rewards.numpy())
    print(rewards_list)
    total_num_ones += np.min(num_ones)
  return total_num_ones / num_episodes

In [ ]:
log_interval = 100
eval_interval = 100

num_train_steps = 100
df = pd.DataFrame()

dataset = replay_buffer.as_dataset(
    sample_batch_size=10,
    num_steps=2)
iterator = iter(dataset)

for step in range(num_train_steps):
  trajectories, _ = next(iterator)
  train_loss = agent.train(experience=trajectories).loss

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))
  if step % eval_interval == 0:
    min_ones = compute_min_ones(tf_eval_env, agent.collect_policy, eval_num_ones)
    print('step = {0}: min_ones = {1}'.format(step, min_ones))
  df = pd.concat([df, pd.DataFrame({'step': [step], 'min_ones': [min_ones], 'loss': [train_loss.numpy()]})])
print(df)

In [ ]:
print(agent.policy)

In [ ]:
df.set_index('step', inplace=True)
df['min_ones'].plot()